In [11]:
# get ratios for segment analysis 
# use BL of whole campaign 
# date created: 3/2/2020
# author: sofia chelpon 


In [1]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [2]:
#### ------------------- LOAD IN 
# data overall 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df_twp.pkl")

# segment data
awas_segments = pd.read_pickle("./awas_segments.pkl")
awas_segments = awas_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON').drop(columns = 'Notes')
# segment data 
toga_segments = pd.read_pickle("./toga_segments.pkl")
toga_segments = toga_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON')

In [3]:
# save positions of each segment 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df_replace_twp.pkl")
awas_df_ut = awas_df.loc[(awas_df['GGALT'] < 14000) & (awas_df['GGALT'] > 12000)]

awas_positions = awas_df_ut.transpose()
awas_positions = awas_positions[:7]
awas_positions.to_pickle("./awas_positions.pkl")

In [4]:
##### --------------------- get campaign average BL means 
# AWAS 
awas_bl = pd.Series()
awas_bl = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()
awas_bl = awas_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_bl.index.name = "Trace_Gas"

# TOGA
toga_bl = pd.Series()
toga_bl = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()
toga_bl = toga_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_bl.index.name = "Trace_Gas"

/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':


In [5]:
######## ------------- GET AWAS RATIOS 
# thing to fill with ratios 
awas_seg_ratios = pd.DataFrame()
for col in awas_df.columns: 
    awas_seg_ratios[col] = ''
awas_seg_ratios = awas_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(awas_segments))):
    # find segment, get info 
    awas_ut = awas_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': awas_ut['Time_UTC'], 'Flight': awas_ut['Flight']})
    # get ratio 
    awas_ut = awas_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = awas_ut/awas_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    awas_seg_ratios = awas_seg_ratios.append(to_append, ignore_index = True)

In [6]:
######## ------------- GET TOGA RATIOS 
# thing to fill with ratios 
toga_seg_ratios = pd.DataFrame()
for col in toga_df.columns: 
    toga_seg_ratios[col] = ''
toga_seg_ratios = toga_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(toga_segments))):
    # find segment, get info 
    toga_ut = toga_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': toga_ut['Time_UTC'], 'Flight': toga_ut['Flight']})
    # get ratio 
    toga_ut = toga_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = toga_ut/toga_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    toga_seg_ratios = toga_seg_ratios.append(to_append, ignore_index = True)


In [7]:
### add attribute to each species (BL, tropo, and UT lifetimes)
fill = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], columns=['Trace_Gas', 'BL_tau', 'TROPO_tau', 'UT_tau'])
fill = fill.append(fill)

# --------- AWAS 
# read in from xls file to make dataframe - awas
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_08042020.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
awas_lifetimes = fill.append(awas_lifetimes)
awas_lifetimes = awas_lifetimes.reset_index().drop('index', axis = 1)

# --------- TOGA
# read in from xls file to make dataframe - toga 
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_08042020.xlsx'
toga_lifetimes = pd.read_excel(mypath2)
toga_lifetimes = fill.append(toga_lifetimes)
toga_lifetimes = toga_lifetimes.reset_index().drop('index', axis = 1)

In [8]:
# combine data from both instruments to one dataframe 
# --------- AWAS 
awas_seg_ratios = awas_seg_ratios.transpose()
awas_seg_ratios.insert(0, 'Instrument', 'AWAS')

# add lifetimes 
awas_seg_ratios.insert(1, 'BL_tau', awas_lifetimes['BL_tau'].values)
awas_seg_ratios.insert(2, 'TROPO_tau', awas_lifetimes['TROPO_tau'].values)
awas_seg_ratios.insert(3, 'UT_tau', awas_lifetimes['UT_tau'].values)

# --------- TOGA 
toga_seg_ratios = toga_seg_ratios.transpose()
toga_seg_ratios.insert(0, 'Instrument', 'TOGA')

# add lifetimes 
toga_seg_ratios.insert(1, 'BL_tau', toga_lifetimes['BL_tau'].values)
toga_seg_ratios.insert(2, 'TROPO_tau', toga_lifetimes['TROPO_tau'].values)
toga_seg_ratios.insert(3, 'UT_tau', toga_lifetimes['UT_tau'].values)

In [9]:
# combine data from both instruments to one dataframe 
toga_seg_ratios = toga_seg_ratios.drop('Time_UTC', axis = 0)
toga_seg_ratios = toga_seg_ratios.drop('Flight', axis = 0)

ratios_comb = awas_seg_ratios.append(toga_seg_ratios)
ratios_comb

,Instrument,BL_tau,TROPO_tau,UT_tau,0,1,2,3,4,5,...,157,158,159,160,161,162,163,164,165,166
Time_UTC,AWAS,NaN,NaN,NaN,2014-01-22 01:44:00,2014-01-22 01:55:15,2014-01-22 02:06:30,2014-01-22 02:17:45,2014-01-22 02:29:00,2014-01-22 02:37:56,...,2014-02-22 01:15:43,2014-02-22 01:24:44,2014-02-22 01:33:43,2014-02-22 01:42:44,2014-02-22 01:51:44,2014-02-24 23:36:58,2014-02-24 23:51:51,2014-02-25 00:06:52,2014-02-25 00:21:51,2014-02-25 05:19:51
Flight,AWAS,NaN,NaN,NaN,RF05,RF05,RF05,RF05,RF05,RF05,...,RF14,RF14,RF14,RF14,RF14,RF15,RF15,RF15,RF15,RF15
C2H6_Ethane,AWAS,18.996560,51.867565,170.000000,0.50874,0.513608,0.438149,0.418676,0.676697,0.481964,...,0.602455,0.586633,0.585416,0.559857,0.556206,0.651138,0.682782,0.598804,0.600021,0.612192
C2H2_Ethyne,AWAS,5.460657,9.761061,18.790019,0.313679,0.346698,0.28066,0.28066,0.619104,0.321934,...,0.387972,0.412736,0.420991,0.387972,0.387972,0.487028,0.495283,0.487028,0.520047,0.396226
C3H8_Propane,AWAS,4.276704,9.379963,22.541651,0.153034,0.173903,0.104342,0.104342,0.612137,0.194771,...,0.215639,0.208683,0.201727,0.15999,0.180859,0.243464,0.2852,0.25042,0.236508,0.236508
C4H10_Isobutane,AWAS,2.193483,4.101641,7.916587,0.0730395,0.0929594,0.0597596,0.0597596,0.683915,0.139439,...,0.106239,0.0863194,0.106239,0.0597596,0.106239,0.132799,0.112879,0.0796795,0.132799,0.119519
C4H10_nButane,AWAS,1.979710,3.901968,8.096203,0.0622095,0.0912406,0.103683,0.0787987,0.67601,0.15345,...,0.0870933,0.0663568,0.0870933,0.0870933,0.0663568,0.103683,0.157597,0.141008,0.132714,0.132714
C5H12_nPentane,AWAS,1.225894,2.360346,4.742530,NaN,NaN,0.0478983,NaN,0.502933,0.0239492,...,0.0478983,NaN,NaN,NaN,NaN,NaN,0.0239492,0.191593,0.0478983,NaN
C6H6_Benzene,AWAS,3.819659,6.602596,12.137543,0.171265,0.182435,0.134034,0.13031,0.584536,0.193604,...,0.264344,0.242005,0.286683,0.256898,0.260621,0.316468,0.349977,0.338807,0.320191,0.27179
OCS_Carbonyl_sulfide,AWAS,2000.000000,2000.000000,2000.000000,1.00398,0.989348,1.00398,1.02409,1.02592,1.01495,...,0.996663,1.00032,0.996663,0.967403,0.938144,0.983862,0.971061,0.954602,0.95826,1.03141


In [10]:
# pickle this!! 
ratios_comb.to_pickle("./contrast_ratios_ut_seg.pkl")

In [11]:
##############################################################################################
##############################################################################################

In [12]:
# save tracer names 
# sort 
ratios_sort = ratios_comb
ratios_sort = ratios_sort.drop('Time_UTC').drop('Flight')
ratios_sort = ratios_sort.sort_values(['TROPO_tau'])

# save ordered bl tau as a variable 
tau = ratios_sort['TROPO_tau']

# drop uncecessary columns 
ratios_sort = ratios_sort.drop(columns = ['Instrument', 'BL_tau', 'TROPO_tau', 'UT_tau'])
ratios_sort

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
Acetaldehyde,0.0129286,0.0129286,0.0129286,0.0129286,NaN,0.0129286,NaN,NaN,0.0129286,0.0129286,...,0.0817088,0.0129286,0.0129286,0.0129286,0.12851,0.146869,NaN,NaN,NaN,0.0129286
C2H6S_Dimethyl_sulfide,0.00604826,0.00604826,NaN,NaN,NaN,NaN,0.0024193,0.0024193,0.0024193,0.00604826,...,0.0181448,0.00604826,0.0024193,0.0120965,NaN,NaN,NaN,NaN,0.00483861,NaN
C5H12_nPentane,NaN,NaN,0.0478983,NaN,0.502933,0.0239492,NaN,0.814272,0.0239492,0.119746,...,0.0478983,NaN,NaN,NaN,NaN,NaN,0.0239492,0.191593,0.0478983,NaN
C4H9NO3_n_Butyl_nitrate,0.0215794,0.0215794,0.0215794,0.0215794,0.428591,0.0929113,0.0749284,0.173834,0.0215794,0.257754,...,0.00359656,0.00359656,0.00359656,0.00359656,0.00359656,0.0329685,0.0539485,0.0239771,0.00899141,0.0599427
n_Butane,0.0472457,0.0640485,0.014826,0.0802583,0.779257,0.235042,0.238007,0.25995,0.198867,0.25738,...,0.0935029,0.12355,0.0838166,0.047641,0.0954797,0.0652346,NaN,NaN,NaN,0.152412
C4H10_nButane,0.0622095,0.0912406,0.103683,0.0787987,0.67601,0.15345,0.157597,0.352521,0.15345,0.203218,...,0.0870933,0.0663568,0.0870933,0.0870933,0.0663568,0.103683,0.157597,0.141008,0.132714,0.132714
i_Butane,0.122868,0.0738374,0.0218886,0.124911,0.759095,0.330371,0.296517,0.246027,0.236396,0.277547,...,0.12958,0.154971,0.19437,0.0922238,0.0895972,0.328328,NaN,NaN,NaN,0.201667
C4H10_Isobutane,0.0730395,0.0929594,0.0597596,0.0597596,0.683915,0.139439,0.0199199,0.225758,0.126159,0.239038,...,0.106239,0.0863194,0.106239,0.0597596,0.106239,0.132799,0.112879,0.0796795,0.132799,0.119519
C4H9NO3_2_Butyl_nitrate,0.471133,0.548483,0.379719,0.457069,0.696151,0.450037,0.330496,0.759438,0.400814,0.99852,...,0.105477,0.0773501,0.0984456,0.0914138,0.133605,0.112509,0.133605,0.105477,0.0632865,0.520355
C6H6_Benzene,0.171265,0.182435,0.134034,0.13031,0.584536,0.193604,0.219666,0.268067,0.230836,0.27179,...,0.264344,0.242005,0.286683,0.256898,0.260621,0.316468,0.349977,0.338807,0.320191,0.27179


In [13]:
# make a thing to fill with tracer names 
trcnames_allseg = pd.DataFrame()
for col in ratios_sort.columns: 
    trcnames_allseg[col] = ''
trcnames_allseg

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166


In [14]:
# loop through each segment 
col = list(ratios_sort)
for i in col:
    utbl_full = np.array(ratios_sort[i].values, dtype=np.float64)
    utbl_not_null_idx = np.argwhere(~np.isnan(utbl_full))
    # ----- inputs without nans 
    my_utbl = utbl_full[utbl_not_null_idx]
    tracer_names = ratios_sort.index[utbl_not_null_idx]
    # ----- fill to make mu*, tau all length 52
    diffa = len(utbl_full) - len(my_utbl)
    if (diffa > 0):
        filla = np.empty((1,diffa))
        filla.fill(np.nan)
        my_utbl = np.append(my_utbl, filla)
        tracer_names = np.append(tracer_names, filla)
    else: 
        #tracer_names = tracer_names.values 
        tracer_names = tracer_names
        #tracer_names = np.ones([52])
        #tracer_names[:] = np.NaN
    # ----- save outputs of tracer names 
    trcnames_allseg[i] = tracer_names

/Users/schelpon/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Support for multi-dimensional indexing (e.g. `index[:, None]`) on an Index is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  


In [15]:
trcnames_allseg.to_pickle("./trcnames_allseg_trtau.pkl")